In [36]:
import pandas as pd
import os
from scipy.sparse import csr_matrix
import sklearn as sk
from sklearn.metrics.pairwise import pairwise_distances
import numpy as np

In [2]:
os.getcwd()

'C:\\Users\\enabi\\OneDrive\\Desktop\\11182022\\DS_apply\\Projects_for_github\\Recommonder_system\\Recommender_system'

In [3]:
df_user_ratedmovies_timestamps= pd.read_csv('all/user_ratedmovies-timestamps.dat', sep='\t', encoding='ISO-8859-1')

In [4]:
df_user_ratedmovies_timestamps.head()

,userID,movieID,rating,timestamp
0,75,3,1.0,1162160236000
1,75,32,4.5,1162160624000
2,75,110,4.0,1162161008000
3,75,160,2.0,1162160212000
4,75,163,4.0,1162160970000


In [5]:
# Obtain the ratings corresponding to some user, More specifically, use the following user IDs: 75, 14454, and 22035

df_user_ratedmovies_timestamps.loc[df_user_ratedmovies_timestamps['userID'].isin([75, 14454, 22035]),:]


,userID,movieID,rating,timestamp
0,75,3,1.0,1162160236000
1,75,32,4.5,1162160624000
2,75,110,4.0,1162161008000
3,75,160,2.0,1162160212000
4,75,163,4.0,1162160970000
...,...,...,...,...
262698,22035,7044,4.0,1106800437000
262699,22035,8207,4.0,1111441710000
262700,22035,8830,0.5,1106800996000
262701,22035,30894,4.0,1106663752000


In [6]:
# Obtain the items rated by the previous users.

df_movies= pd.read_csv('all/movies.dat', sep='\t', encoding='ISO-8859-1')

In [7]:
df_movies.head()

,id,title,imdbID,spanishTitle,imdbPictureURL,year,rtID,rtAllCriticsRating,rtAllCriticsNumReviews,rtAllCriticsNumFresh,...,rtAllCriticsScore,rtTopCriticsRating,rtTopCriticsNumReviews,rtTopCriticsNumFresh,rtTopCriticsNumRotten,rtTopCriticsScore,rtAudienceRating,rtAudienceNumRatings,rtAudienceScore,rtPictureURL
0,1,Toy story,114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995,toy_story,9,73,73,...,100,8.5,17,17,0,100,3.7,102338,81,http://content7.flixster.com/movie/10/93/63/10...
1,2,Jumanji,113497,Jumanji,http://ia.media-imdb.com/images/M/MV5BMzM5NjE1...,1995,1068044-jumanji,5.6,28,13,...,46,5.8,5,2,3,40,3.2,44587,61,http://content8.flixster.com/movie/56/79/73/56...
2,3,Grumpy Old Men,107050,Dos viejos gruñones,http://ia.media-imdb.com/images/M/MV5BMTI5MTgy...,1993,grumpy_old_men,5.9,36,24,...,66,7,6,5,1,83,3.2,10489,66,http://content6.flixster.com/movie/25/60/25602...
3,4,Waiting to Exhale,114885,Esperando un respiro,http://ia.media-imdb.com/images/M/MV5BMTczMTMy...,1995,waiting_to_exhale,5.6,25,14,...,56,5.5,11,5,6,45,3.3,5666,79,http://content9.flixster.com/movie/10/94/17/10...
4,5,Father of the Bride Part II,113041,Vuelve el padre de la novia (Ahora también abu...,http://ia.media-imdb.com/images/M/MV5BMTg1NDc2...,1995,father_of_the_bride_part_ii,5.3,19,9,...,47,5.4,5,1,4,20,3,13761,64,http://content8.flixster.com/movie/25/54/25542...


In [8]:
# Obtain the items rated by the previous users.

In [9]:
df_user_ratedmovies_timestamps.loc[df_user_ratedmovies_timestamps['userID'].isin([75, 14454, 22035]), 'movieID']

0             3
1            32
2           110
3           160
4           163
          ...  
262698     7044
262699     8207
262700     8830
262701    30894
262702    49272
Name: movieID, Length: 1188, dtype: int64

In [10]:
list(df_user_ratedmovies_timestamps.loc[df_user_ratedmovies_timestamps['userID'].isin([75, 14454, 22035]), 'movieID'])

[3,
 32,
 110,
 160,
 163,
 165,
 173,
 296,
 353,
 420,
 589,
 653,
 832,
 920,
 996,
 1036,
 1127,
 1215,
 1233,
 1304,
 1370,
 1374,
 1485,
 1527,
 1917,
 2011,
 2054,
 2058,
 2490,
 2571,
 2640,
 2688,
 2700,
 2762,
 2959,
 3258,
 3793,
 3889,
 3994,
 4993,
 5107,
 5833,
 5952,
 6213,
 6225,
 6333,
 7000,
 7007,
 7153,
 32029,
 32587,
 33437,
 39052,
 45431,
 45722,
 1,
 16,
 18,
 19,
 21,
 29,
 32,
 36,
 39,
 42,
 45,
 47,
 50,
 65,
 68,
 69,
 70,
 88,
 95,
 101,
 107,
 110,
 111,
 112,
 117,
 145,
 150,
 153,
 157,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 173,
 175,
 176,
 180,
 183,
 185,
 194,
 196,
 198,
 213,
 223,
 231,
 233,
 235,
 246,
 247,
 253,
 260,
 266,
 268,
 288,
 290,
 292,
 293,
 296,
 300,
 308,
 318,
 319,
 326,
 337,
 344,
 349,
 350,
 352,
 353,
 354,
 355,
 356,
 360,
 364,
 367,
 371,
 372,
 373,
 376,
 377,
 380,
 382,
 410,
 413,
 423,
 427,
 432,
 434,
 435,
 436,
 440,
 441,
 442,
 444,
 450,
 454,
 455,
 456,
 457,
 461,
 466,
 474,
 475,
 480,
 482,

In [11]:
df_movies.loc[df_movies['id'].isin(list(df_user_ratedmovies_timestamps.loc[df_user_ratedmovies_timestamps['userID'].isin([75, 14454, 22035]), 'movieID'])) ,'title']

0                                        Toy story
2                                   Grumpy Old Men
15                                          Casino
17                                      Four Rooms
18                  Ace Ventura: When Nature Calls
                           ...                    
8964                    Zathura: A Space Adventure
8987                                     King Kong
9135                                Over the Hedge
9154    Pirates of the Caribbean: Dead Man's Chest
9323                                 Casino Royale
Name: title, Length: 1048, dtype: object

In [12]:
# Obtain the items rated by the user 75

In [13]:
# Obtain the id of the items rated by the user 75
df_user_ratedmovies_timestamps.loc[df_user_ratedmovies_timestamps['userID']==75, 'movieID']

0         3
1        32
2       110
3       160
4       163
5       165
6       173
7       296
8       353
9       420
10      589
11      653
12      832
13      920
14      996
15     1036
16     1127
17     1215
18     1233
19     1304
20     1370
21     1374
22     1485
23     1527
24     1917
25     2011
26     2054
27     2058
28     2490
29     2571
30     2640
31     2688
32     2700
33     2762
34     2959
35     3258
36     3793
37     3889
38     3994
39     4993
40     5107
41     5833
42     5952
43     6213
44     6225
45     6333
46     7000
47     7007
48     7153
49    32029
50    32587
51    33437
52    39052
53    45431
54    45722
Name: movieID, dtype: int64

In [14]:
# Obtain the name of the items rated by the user 75
df_movies.loc[df_movies['id'].isin(list(df_user_ratedmovies_timestamps.loc[df_user_ratedmovies_timestamps['userID']==75, 'movieID'])) ,'title']

2                                          Grumpy Old Men
31                                         Twelve Monkeys
105                                            Braveheart
151                                                 Congo
154                                             Desperado
156                            Die Hard: With a Vengeance
164                                           Judge Dredd
285                                          Pulp Fiction
340                                              The Crow
403                                 Beverly Hills Cop III
566                            Terminator 2: Judgment Day
623                                           DragonHeart
766                                                Ransom
839                                    Gone with the Wind
909                                     Last Man Standing
949                                              Die Hard
1028                                            The Abyss
1103          

In [15]:
# Obtain the items rated by the user 14454

In [16]:
# Obtain the id of the items rated by the user 14454
df_user_ratedmovies_timestamps.loc[df_user_ratedmovies_timestamps['userID']==14454, 'movieID']

169927        1
169928       16
169929       18
169930       19
169931       21
          ...  
170775    38886
170776    39292
170777    39381
170778    40851
170779    41569
Name: movieID, Length: 853, dtype: int64

In [17]:
# Obtain the name of the items rated by the user 14454
df_movies.loc[df_movies['id'].isin(list(df_user_ratedmovies_timestamps.loc[df_user_ratedmovies_timestamps['userID']==14454, 'movieID'])) ,'title']

0                            Toy story
15                              Casino
17                          Four Rooms
18      Ace Ventura: When Nature Calls
20                          Get Shorty
                     ...              
8904           The Squid and the Whale
8911        Good Night, and Good Luck.
8914                   The Proposition
8964        Zathura: A Space Adventure
8987                         King Kong
Name: title, Length: 853, dtype: object

In [18]:
# Obtain the items rated by the user 22035

In [19]:
# Obtain the id of the items rated by the user 22035
df_user_ratedmovies_timestamps.loc[df_user_ratedmovies_timestamps['userID']==22035, 'movieID']

262423       24
262424       25
262425       32
262426       36
262427       50
          ...  
262698     7044
262699     8207
262700     8830
262701    30894
262702    49272
Name: movieID, Length: 280, dtype: int64

In [20]:
# Obtain the name of the items rated by the user 22035
df_movies.loc[df_movies['id'].isin(list(df_user_ratedmovies_timestamps.loc[df_user_ratedmovies_timestamps['userID']==22035, 'movieID'])) ,'title']

23                                        Powder
24                             Leaving Las Vegas
31                                Twelve Monkeys
35                              Dead Man Walking
49                            The Usual Suspects
                          ...                   
6654                               Wild at Heart
7383                       The Day of the Jackal
7801    Anacondas: The Hunt for the Blood Orchid
8446                                 White Noise
9323                               Casino Royale
Name: title, Length: 280, dtype: object

In [21]:
#Create a sparse matrix based on the previous information, This matrix will include each user as a row, and the items will be the columns
df_user = df_user_ratedmovies_timestamps.loc[df_user_ratedmovies_timestamps['userID'].isin([75, 14454, 22035]),:]

In [22]:
df_user

,userID,movieID,rating,timestamp
0,75,3,1.0,1162160236000
1,75,32,4.5,1162160624000
2,75,110,4.0,1162161008000
3,75,160,2.0,1162160212000
4,75,163,4.0,1162160970000
...,...,...,...,...
262698,22035,7044,4.0,1106800437000
262699,22035,8207,4.0,1111441710000
262700,22035,8830,0.5,1106800996000
262701,22035,30894,4.0,1106663752000


In [23]:
matrix = df_user.pivot(index='userID', columns='movieID', values='rating')

In [24]:
matrix

movieID,1,3,16,18,19,21,24,25,29,32,...,38038,38886,39052,39292,39381,40851,41569,45431,45722,49272
userID,,,,,,,,,,,,,,,,,,,,,
75,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,...,NaN,NaN,3.5,NaN,NaN,NaN,NaN,3.5,4.5,NaN
14454,2.5,NaN,3.0,1.0,1.0,3.0,NaN,NaN,3.5,3.5,...,3.0,3.5,NaN,2.5,3.5,2.0,3.0,NaN,NaN,NaN
22035,NaN,NaN,NaN,NaN,NaN,NaN,2.0,4.0,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0


In [25]:
# creating the same matrix while the name of the movies are replaced by the id of movies:

new_columns = dict(zip(df_movies['id'], df_movies['title']))
df1 = matrix.rename(columns=new_columns)

In [26]:
df1

movieID,Toy story,Grumpy Old Men,Casino,Four Rooms,Ace Ventura: When Nature Calls,Get Shorty,Powder,Leaving Las Vegas,La cité des enfants perdus,Twelve Monkeys,...,Wallace & Gromit in The Curse of the Were-Rabbit,The Squid and the Whale,Star Wreck: In the Pirkinning,"Good Night, and Good Luck.",The Proposition,Zathura: A Space Adventure,King Kong,Over the Hedge,Pirates of the Caribbean: Dead Man's Chest,Casino Royale
userID,,,,,,,,,,,,,,,,,,,,,
75,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,...,NaN,NaN,3.5,NaN,NaN,NaN,NaN,3.5,4.5,NaN
14454,2.5,NaN,3.0,1.0,1.0,3.0,NaN,NaN,3.5,3.5,...,3.0,3.5,NaN,2.5,3.5,2.0,3.0,NaN,NaN,NaN
22035,NaN,NaN,NaN,NaN,NaN,NaN,2.0,4.0,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0


In [27]:
matrix_zero= df1.copy()

In [34]:
# Compute a similarity function based on the user information encoded in the previous matrix.
# cosine_similarity from the sklearn library

matrix_zero.fillna(0, inplace=True)
Matt =matrix_zero.values
sparse_matrix = csr_matrix(Matt)
sk.metrics.pairwise.cosine_similarity(sparse_matrix, Y=None, dense_output=True)

array([[1.        , 0.09606587, 0.12122035],
       [0.09606587, 1.        , 0.24952283],
       [0.12122035, 0.24952283, 1.        ]])

In [37]:
# implement your own version of this similarity function


# calculate dot product between rows of the sparse matrix
dot_products = sparse_matrix.dot(sparse_matrix.T)

# calculate Euclidean norm of rows of the sparse matrix
row_norms = np.sqrt(np.sum(np.square(Matt), axis=1))

# calculate cosine similarity
cosine_similarity = dot_products.multiply(1 / row_norms).T.multiply(1 / row_norms)


In [39]:
print(cosine_similarity)

  (0, 0)	1.0
  (0, 1)	0.09606587350870287
  (0, 2)	0.12122035158657601
  (1, 0)	0.09606587350870285
  (1, 1)	0.9999999999999999
  (1, 2)	0.2495228275432249
  (2, 0)	0.12122035158657601
  (2, 1)	0.24952282754322494
  (2, 2)	1.0


In [40]:
# Jaccard or correlation

# Jaccard
binary_matrix = matrix.notnull().astype('int')

# calculate Jaccard similarity matrix
jaccard_sim_matrix = 1 - pairwise_distances(binary_matrix.values, metric='jaccard')

C:\Users\enabi\anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:1875: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [41]:
jaccard_sim_matrix

array([[1.        , 0.0259887 , 0.046875  ],
       [0.0259887 , 1.        , 0.10644531],
       [0.046875  , 0.10644531, 1.        ]])

In [42]:
# Compute the correlation matrix using pairwise_distances
correlation_matrix = pairwise_distances(sparse_matrix.toarray(), metric='correlation')

In [43]:
correlation_matrix

array([[0.        , 1.18640365, 0.98442244],
       [1.18640365, 0.        , 1.3967668 ],
       [0.98442244, 1.3967668 , 0.        ]])